In [19]:
import json

In [ ]:
import re
f = open('search-terms.txt','r')
p = '|(\s|#|@)'.join([line.rstrip() for line in f.readlines()])
f.close()


In [40]:
rdd = sc.textFile('data/2015-09-16-20-00.gz').cache()

In [106]:
def make_json(tweet):
    ''' Get stringified JSOn from Kafka, attempt to convert to JSON '''
    try:
        return json.loads(tweet.decode('utf-8'))
    except:
        return "error"+str(tweet.decode('utf-8'))

def filter_tweets(item,pattern):
    ''' Filters out the tweets we do not want.  Filters include:
            * No retweets 
            * No geolocation or location field (do we really care about this?)
            * English language only
            * No tweets with links
                - We need to check both entities and media fields for this (is that true?)
    '''
    return (('delete' not in item.keys()) and
            ('retweeted_status' not in item.keys())                           and 
            #((item['geo'] is not None) or (item['user']['location'] is not None)) and
            (item['lang']=='en')                       and
            (len(item['entities']['urls'])==0)                   and
            ('media' not in item['entities'].keys()) and
            (re.search(pattern,item['text'],re.I) is not None)
           )

def get_relevant_fields(item):
    ''' Reduce the full set of metadata down to only those we care about, including:
            * timestamp
            * username
            * text of tweet 
            * hashtags
            * geotag coordinates (if any)
            * location (user-defined in profile, not necessarily current location)
    '''
    return (item['id'], 
            {"timestamp":  item['created_at'],
             "username":    item['user']['screen_name'],
             "text":        item['text'],
             "hashtags":    [el['text'] for el in item['entities']['hashtags']],
             "geotag":    item['geo'],
             "user_loc":    item['user']['location']
            }
           )

In [42]:
print "Total:",rdd.count()

print "Non-deletes:",rdd.map(make_json).count()

print "Passed filter:",rdd.map(make_json).filter(filter_tweets).count()


Total: 452684
Non-deletes: 409082
Passed filter: 409082


In [108]:
rddj = rdd.map(make_json).cache()

In [ ]:
print "Passed filter:",rdd.map(make_json).filter(filter_tweets).count()

In [116]:
a = rddj.filter(lambda x: filter_tweets(x,p)).map(get_relevant_fields)

In [118]:
postf = a.collect()


In [ ]:
for x in postf:
    print x['created_at']

In [104]:
def terms_filter(item,p):
    return re.search(p,item['text'],re.I)

In [105]:
for tw in a:
    rgx = re.search(p,tw[1]['text'],re.I)
    if rgx is not None:
        print "MATCH:", tw[1]['text']
        print

MATCH: Thoughts &amp; prayers go out to all the closeted liberal teens who will be watching the #GOPDebate tonight w/ their conservative families.

MATCH: #RatPackReps Estbmnt pols: @BobbyJindal @lindsaygrahamsc Petaki whose goal is to defeat #Trump by lying. Am is dying bec of these jackasses

MATCH: Going get good laugh tonight..watching these idiots. Too bad @pmharper couldn't  join them; he'll fit right in. #CNNDebate #elxn42

MATCH: #CNNDebate...the worst part of this debate is that it is on CNN...

MATCH: @alltookayla Bernie is gonna drag duh

MATCH: @talkwordy @HillaryJacques Now I want a kitkat. Damn you!

MATCH: Candidates getting psyched to music for debate:  Christie = Jersey Boys Soundtrack. #MakeAmericaGreatAgain

MATCH: As an outsider (and I'm not American citizen) I really feel Trump makes this election more like his reality show. Politics=fun! #GOPDebate

MATCH: @BobbyJindal is pissed he's so sorry. Definitely little man syndrome! #CNNDebate

MATCH: JV #GOP #CNNDebate c

In [91]:
p

"gopdebate|cnndebate|cnbcdebate|foxbusinessdebate|republicandebate|democraticdebate|candidates|clinton|clinton's|hillary|hillary's|hillaryclinton|readyforhillary|Hillary2016|sanders|sanders'|sanders's|bernie|bernie's|feelthebern|BernieSanders2016|Sanders2016|omalley|OMalley2016|martinomalley|bush|bush's|jeb|jeb's|jebbush|jebcanfixit|AllinForJeb|carson|carson's|bencarson|BC2DC16|ImWithBen|Carson2016|cruz|cruz's|tedcruz|ATimeForTruth|CruzCrew|fiorina|fiorina's|carlyfiorina|Carly2016|kaisch|kaisch's|johnkaisch|rand|rand's|randpaul|StandWithRand|rubio|rubio's|marcorubio|TeamMarco|trump|trump's|donaldtrump|DonaldTrumpforPresident|Trump2016|makeamericagreatagain|realdonaldtrump"